<a id=toc></a>

# <h1 style="padding: 35px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;background-position: 0px 0px; "><span style='color:white'><b>NLP | Sentiment Analysis of Company Reviews</b></span></h1>


<center>
    <figure>
        <img src="https://www.surveysensum.com/wp-content/uploads/2020/02/SENTIMENT-09-1.png" alt ="Sentiment Analysis" style='width:80%;'>
        <figcaption>
            Source: <a href="https://www.surveysensum.com/customer-experience/sentiment-analysis/">[surveysensum.com | Sentiment Analysis: A Thorough Guide For The Data Geek]</a>
        </figcaption>
    </figure>
</center>

### 🎯 Objective

The objective for this notebook is to build a baseline model which is capable of predicting the sentiment of company reviews left by customers for the [Sentiment Analysis - Company Reviews Competition](https://www.kaggle.com/competitions/sentiment-analysis-company-reviews/overview).

### 🤔 So what exactly is Sentiment Analysis?

**According to [ChatGPT](https://openai.com/blog/chatgpt/):**

> Sentiment analysis, also known as opinion mining, is the process of analyzing and identifying the sentiment, attitude, or emotion expressed in a piece of text, such as a review or a social media post.
>
> It is commonly used in various industries, such as marketing, customer service, and politics, to understand people's opinions, preferences, and behavior. Sentiment analysis uses natural language processing (NLP) techniques and algorithms to analyze and classify text into different categories, such as positive, negative, or neutral.The process involves pre-processing the text, such as tokenization and stemming, to convert it into a format that can be analyzed by the algorithms.
>
>Sentiment analysis models can be trained on a dataset of labeled data, which contains examples of text with their corresponding sentiment labels. Alternatively, models can be trained using unsupervised learning techniques, which use clustering and other methods to classify the text without labeled data.
>
>Sentiment analysis can be performed on various types of text, such as social media posts, product reviews, news articles, and customer feedback. Overall, sentiment analysis is a useful tool for understanding people's opinions and attitudes, and can help businesses make data-driven decisions based on customer feedback.

**For more information on Sentiment Analysis, see the following links:**
> - [MonkeyLearn | Sentiment Analysis: A Definitive Guide](https://monkeylearn.com/sentiment-analysis/)
> - [Thematic | Sentiment Analysis: Comprehensive Beginners Guide](https://getthematic.com/sentiment-analysis/)

### 📁 Dataset
The dataset for this competition (both train and test) consists of 100,000 reviews collected from Trustpilot and spans over 40 different companies.

> Find this competitions dataset here: [Sentiment Analysis - Company Reviews Dataset](https://www.kaggle.com/competitions/sentiment-analysis-company-reviews/data)

<br>

## Table of contents

- [1 | Dataset Exploration](#1)
   > - [Load CSV Files](#1.1)
   > - [View Random Selected Samples](#1.2)
   > - [View Train Rating Distribution](#1.3)
   > - [Inspect Review Lengths & Tokens](#1.4)
   > - [View Review Lengths & Review Token Count Histograms](#1.5)
  
- [2 | Data Preprocessing](#2)
   > - [Label Encode Ratings](#2.1)
   > - [Create Train/Validation Split](#2.2)
   > - [View New Train & Validation Labels Distribution](#2.3)
   
- [3 | Build Input Data Pipeline with tf.data API](#3)
   > - [Define Text Preprocessor](#3.1)
   > - [Generate Input Data Pipelines](#3.2)
   
- [4 | Baseline Model: Universal Sentence Encoder Model](#4)
   > - [TensorFlow Hub](#tfhub)
   > - [Get Universal Sentence Encoder](#4.1)
   > - [Build Model](#4.2)

- [5 | Train Baseline Model](#5)
   > - [Define Callbacks and Metrics for Model Training](#5.1)
   > - [Compile & Train Model](#5.2)
   
- [6 | Model Performance Evaluation](#6)
   > - [View Model Histories](#6.1)
   > - [Plot Confusion Matrix](#6.2)
   > - [Generate Classification Reports](#6.3)
   > - [Record Classification Metrics](#6.4)
   
- [7 | Generate Submission](#7)
   > - [Preprocess Test Reviews](#7.1)
   > - [Generate Test Predictions](#7.2)
   > - [Generate Submission.csv](#7.3)
   
- [Conclusion](#conclusion)

<br>
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>

In [ ]:
import re
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import top_k_accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix, mean_absolute_error
from scikitplot.metrics import plot_roc

In [ ]:
warnings.filterwarnings("ignore")
plt.style.use('dark_background')

In [ ]:
class CFG:
    SEED = 768
    BATCH_SIZE = 32
    EPOCHS = 10

In [ ]:
def seed_everything(seed=CFG.SEED):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(CFG.SEED)

<a id='1'></a>
# 1 | Dataset Exploration
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>

In [ ]:
# Define paths
DATASET_PATH = "/kaggle/input/sentiment-analysis-company-reviews/"
TRAIN_CSV = '/kaggle/input/sentiment-analysis-company-reviews/train.csv'
TEST_CSV = '/kaggle/input/sentiment-analysis-company-reviews/test.csv'
SAMPLE_SUB_CSV = '/kaggle/input/sentiment-analysis-company-reviews/sample_submission.csv'

### Load CSV Files

In [ ]:
# Load the csv files
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
submission_df = pd.read_csv(SAMPLE_SUB_CSV) 

In [ ]:
# Generate summary of the training set
train_df.info()

In [ ]:
# View first 5 training samples
train_df.head(5)

<a id='1.2'></a>
### View Random Selected Samples

In [ ]:
def view_samples(df, count=5):
    idx = random.sample(train_df.index.to_list(), count)
    print('=========================================\n')
    for _ in idx:
        print(f'id:\t{df.Id[_]}\n')
        print(f'Review:\n{df.Review[_]}\n')
        print(f'Rating:\n{df.Rating[_]}')
        print('=========================================\n')

In [ ]:
# View 5 randomly selected samples
view_samples(train_df, count=5)

<a id='1.3'></a>
### View Train Rating Distribution

In [ ]:
# View Train Rating Distribution
plt.figure(figsize=(15, 8))
plt.title('Train Rating Distribution', fontsize=20)

train_distribution = train_df['Rating'].value_counts().sort_values()
sns.barplot(x=list(train_distribution.keys()),
            y=train_distribution.values);

sns.despine();

<div class="alert alert-block alert-info">
<br>
We observe that <b>the dataset is severly imbalanced</b> with review with ratings of 1 and 5 make up the majority of the dataset's reviews. Techniques such as undersampling, oversampling or weighted training should be considered. However, this notebook will only focus on the baseline model and will not cover the implementation of these techniques. It should be noted that this behavior in the data is most likely representative of customer behavior (customers would rather give a rating of 1 or 5 and ratings in between siginify mixed opinions). 
</div>

<a id='1.4'></a>
### Inspect Review Lengths & Tokens

In [ ]:
# Get the lengths of each review
train_df['review_length'] = [len(_) for _ in train_df.Review]

# Get the number of tokens per review 
train_df['token_count'] = [len(_.split()) for _ in train_df.Review]

In [ ]:
# View first 5 samples 
train_df.head(5)

In [ ]:
# Inspect Review Length Stats
print('Review Length Description')
print('==================================')
print(train_df['review_length'].describe())
print('==================================')

In [ ]:
# Inspect Token Count Stats
print('Token Count Description')
print('==================================')
print(train_df['token_count'].describe())
print('==================================')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(14, 18))

# Set the spacing between subplots
fig.tight_layout(pad=6.0)

# Plot Range of Review Lengths per Rating
ax1.set_title('Review Lengths per Rating', fontsize=20)
sns.boxplot(data=train_df, y='review_length', x='Rating',
            ax=ax1)
ax1.set_xlabel('Rating', fontsize=14)
ax1.set_ylabel('review_length', fontsize=14)
sns.despine();

# Plot Range of Token Counts per Rating
ax2.set_title('Token Counts per Rating', fontsize=20)
sns.boxplot(data=train_df, y='token_count', x='Rating',
            ax=ax2);
ax2.set_xlabel('Rating', fontsize=14)
ax2.set_ylabel('token_count', fontsize=14)
sns.despine();

<div class="alert alert-block alert-info">
<br>
We observe that the length of the reviews increase the more unsatisfied the customers are with the companies. The same observation can be made for the number of tokens per review. The reason for this may be that customers tend to explain or describe their opinions/experiences in great detail the more unsatisfied they are with the companies. This reason may also explain why reviews with higher ratings are generally shorter with less tokens present as this signifies satification amongst customers.
</div>

<a id='1.5'></a>
### View Review Lengths & Review Token Count Histograms

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(14, 10))

# Set the spacing between subplots
fig.tight_layout(pad=6.0)

# Generate Train Rating Histogram
ax1.set_title('Train Review Length Histogram', fontsize=20)
sns.histplot(data=train_df, x='review_length', bins=50,
            ax=ax1)
ax1.set_xlabel('review_length', fontsize=14)
ax1.set_ylabel('Count', fontsize=14)
sns.despine();

# Generate Train Token Count Histogram
ax2.set_title('Train Token Count Histogram', fontsize=20)
sns.histplot(data=train_df, x='token_count', bins=50,
            ax=ax2)
ax2.set_xlabel('token_count', fontsize=14)
ax2.set_ylabel('Count', fontsize=14)
sns.despine();

<div class="alert alert-block alert-info">
<br>
We observe that the majority of review lengths are under a length of ~1000. We also observe that the review token counts are generally under ~300 tokens. These factors should be considered when selecting the number of tokens to be used in a model. Selecting the number of tokens to be used in model via percentiles may prove to be helpful. However, this will not be covered in this notebook.
</div>

<center><div style='color:#ffffff;
           display:inline-block;
           padding: 5px 5px 5px 5px;
           border-radius:5px;
           background-color:#03e3fc;
           font-size:100%;'><a href=#toc style='text-decoration: none; color:#03001C;'>⬆️ Back To Top</a></div></center>

<a id='2'></a>
# 2 | Data Preprocessing
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>

<a id='2.1'></a>
### Label Encode Ratings

We need to label encode the ratings since all ratings fall in the range between 1 and 5 inclusively. To achieve this we simply shift the ratings by subtracting 1 from each rating (e.g. 5 -> 4). We do this in order to simplify the one-hot encoding process at a later stage.

In [ ]:
# Label encode ratings
train_df["rating_encoded"] = train_df['Rating'] - 1

<a id='2.2'></a>
### Create Train/Validation Split

In [ ]:
# Get indices for train and validation splits
train_idx, val_idx, _, _ = train_test_split(
    train_df.index, train_df.Rating, 
    test_size=0.2, stratify=train_df.Rating,
    random_state=CFG.SEED
)

In [ ]:
# Get new training and validation data
train_new_df = train_df.iloc[train_idx].reset_index(drop=True)
val_df = train_df.iloc[val_idx].reset_index(drop=True)

# View shapes
train_new_df.shape, val_df.shape

In [ ]:
# View new train dataframe
train_new_df

<a id='2.3'></a>
### View Train & Validation Rating Distributions

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(14, 10))

# Set the spacing between subplots
fig.tight_layout(pad=6.0)

# Plot New Train Ratings Distribution
ax1.set_title('New Train Ratings Distribution', fontsize=20)
train_new_distribution = train_new_df['Rating'].value_counts().sort_values()
sns.barplot(x=train_new_distribution.values,
            y=list(train_new_distribution.keys()),
            orient="h",
            ax=ax1)
sns.despine();

# Plot Validation Ratings Distribution
ax2.set_title('Validation Ratings Distribution', fontsize=20)
val_distribution = val_df['Rating'].value_counts().sort_values()
sns.barplot(x=val_distribution.values,
            y=list(val_distribution.keys()),
            orient="h",
            ax=ax2);
sns.despine();

<center><div style='color:#ffffff;
           display:inline-block;
           padding: 5px 5px 5px 5px;
           border-radius:5px;
           background-color:#03e3fc;
           font-size:100%;'><a href=#toc style='text-decoration: none; color:#03001C;'>⬆️ Back To Top</a></div></center>
           
<a id='3'></a>
# 3 | Build Input Data Pipeline with tf.data API
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>
<br>

In this notebook we'll use the tf.data API to build input data pipelines for training a model and conducting model inference. In order to achieve this, we'll preprocess the reviews by removing any artifacts in the texts such as emojis, non-ascii characters and replacing numbers with another character. The preprocessed texts will be used to construct the pipelines along with the one-hot encoded ratings.

<center>
    <figure>
        <img src="https://i.postimg.cc/GpPNLLNH/1-3-Zi3-VUAYh5w78-KGLo-O37g.webp" alt ="UCF101" style='width: 800px;'>
        <figcaption>
            Source: <a href="https://towardsdatascience.com/how-to-reduce-training-time-for-a-deep-learning-model-using-tf-data-43e1989d2961">[Medium | How to Reduce Training Time for a Deep Learning Model using tf.data]</a>
        </figcaption>
    </figure>
</center>

<br>

**For more information on the tf.data API and loading data from generator, follow these links:** 
> - [tf.data: Build TensorFlow input pipelines](https://www.tensorflow.org/guide/data)
> - [Better performance with the tf.data API](https://www.tensorflow.org/guide/data_performance)
> - [Using generators with tf.data API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_generator)

<a id='3.1'></a>
### Define Text Preprocessor

In [ ]:
def text_preprocessor(text):
    
    # -----------------------------------------------------
    # Source: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    # -----------------------------------------------------
    non_ascii_pattern = re.compile(r"[^\x00-\x7F]+", flags=re.UNICODE)
    digit_pattern = re.compile('[0-9]', flags=re.UNICODE)
    
    # -----------------------------------------------------
    # Source: https://stackoverflow.com/questions/21932615/regular-expression-for-remove-link
    link_pattern = re.compile('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', flags=re.UNICODE)
    # -----------------------------------------------------
    
    # Remove emojis
    preprocessed_text = emoji_pattern.sub(r'', text)
    # Remoce non-ascii characters
    preprocessed_text = non_ascii_pattern.sub(r'', preprocessed_text)
    # Replace numbers with '@' sign
    preprocessed_text = digit_pattern.sub(r'#', preprocessed_text)
    # Remove web links 
    preprocessed_text = link_pattern.sub(r'', preprocessed_text)
    
    return preprocessed_text

<a id='3.2'></a>
### Generate Input Data Pipelines

In [ ]:
def encode_labels(labels, label_depth=5):
    return tf.one_hot(labels, depth=label_depth).numpy()

def create_pipeline(df, preprocessor, batch_size=32, shuffle=False, cache=None, prefetch=False):
    '''
    Generates an input pipeline using the tf.data API given a Pandas DataFrame and image loading function.
    
    @params
        - df: (pd.DataFrame) -> DataFrame containing texts and labels
        - preprocessor (function) -> preprocessor used to preprocess texts
        - batch_size: (int) -> size for batched (default=32) 
        - shuffle: (bool) -> condition for data shuffling, data is shuffled when True (default=False)
        - cache: (str) -> cache path for caching data, data is not cached when None (default=None)
        - prefetch: (bool) -> condition for prefeching data, data is prefetched when True (default=False)
        
    @returns
        - dataset: (tf.data.Dataset) -> dataset input pipeline used to train a TensorFlow model
    '''
    # Get image paths and labels from DataFrame
    reviews = df['Review'].apply(preprocessor).to_numpy().astype(str)
    ratings = encode_labels(df['rating_encoded'].to_numpy().astype(np.float32))
    AUTOTUNE = tf.data.AUTOTUNE
    
    # Create dataset with raw data from DataFrame
    ds = tf.data.Dataset.from_tensor_slices((reviews, ratings))
    
    # Apply shuffling based on condition
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
        
    # Apply batching
    ds = ds.batch(batch_size)
    
    # Apply caching based on condition
    # Note: Use cache in memory (cache='') if the data is small enough to fit in memory!!!
    if cache != None:
        ds = ds.cache(cache)
    
    # Apply prefetching based on condition
    # Note: This will result in memory trade-offs
    if prefetch:
        ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    # Return the dataset
    return ds

In [ ]:
# Create train input data pipeline
train_ds = create_pipeline(
    train_new_df, text_preprocessor, 
    batch_size=CFG.BATCH_SIZE, 
    shuffle=False, prefetch=True
)

# Create validation input data pipeline
val_ds = create_pipeline(
    val_df, text_preprocessor,
    batch_size=CFG.BATCH_SIZE, 
    shuffle=False, prefetch=False
)

In [ ]:
# View string representation of datasets
print('========================================')
print('Train Input Data Pipeline:\n\n', train_ds)
print('========================================')
print('Validation Input Data Pipeline:\n\n', val_ds)
print('========================================')

<center><div style='color:#ffffff;
           display:inline-block;
           padding: 5px 5px 5px 5px;
           border-radius:5px;
           background-color:#03e3fc;
           font-size:100%;'><a href=#toc style='text-decoration: none; color:#03001C;'>⬆️ Back To Top</a></div></center>

<a id='4'></a>
# 4 | Baseline Model: Universal Sentence Encoder Model
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>
<br>

The Universal Sentence Encoder encodes text into high dimensional vectors that can be used for text classification, semantic similarity, clustering, and other natural language tasks. For this baseline model we'll make use of Universal Sentence Encoder (USE) to generate embeddings which are representative of the review texts.

<center>
    <figure>
        <img src="https://amitness.com/posts/images/use-overall-pipeline.png" alt ="USE Diagram" style='width: 600px;'>
        <figcaption>
            Source: <a href="https://amitness.com/posts/universal-sentence-encoder">[Amit Chaudhary | Universal Sentence Encoder Visually Explained]</a>
        </figcaption>
    </figure>
</center>

**Reasons for using Universal Sentence Encoder:**
1. Minimal hardware requirements for generating embeddings with USE
2. Low inference rate
3. Light-weight memory consumptions

**Drawbacks of using Universal Sentence Encoder:**
1. Embedding representations become less accurate as text lengths increases.
2. Although USE has a low inference rate, its accuracy falls short when compared to language models such as BERT & DeBERTa-v3

<br>

**For more information regarding the Universal Sentence Encoder, follow these links:** 
> - [An Introduction to Transfer Learning](https://georgian.io/an-introduction-to-transfer-learning/)
> - [A Comprehensive Hands-on Guide to Transfer Learning with Real-World Applications in Deep Learning](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a)
> - <a href="https://amitness.com/2020/06/universal-sentence-encoder/">Amit Chaudhary | Universal Sentence Encoder Visually Explained</a>

<br>

<a id="tfhub"></a>
## TensorFlow Hub

TensorFlow Hub is a repository of trained machine learning models ready for fine-tuning and deployable anywhere. TensorFlow Hub enables us to reuse trained models like BERT and Faster R-CNN with just a few lines of code. In this section we'll get the USE model from TensorFlow Hub. 

**For more information on TensorFlow Hub or if you would like to access the other models in PyTorch/JAX, check out the following links:**
> - [TensorFlow Hub](https://www.tensorflow.org/hub)
> - [HuggingFace🤗](https://huggingface.co/)

In [ ]:
# Here's a function to get any model/preprocessor from tensorflow hub
def get_tfhub_model(model_link, model_name, model_trainable=False):
    return hub.KerasLayer(model_link,
                          trainable=model_trainable,
                          name=model_name)

<a id='4.1'></a>
### Get Universal Sentence Encoder

In [ ]:
# Get Universal Sentence Encoder here
# -----------------------------------
# Note: We'll use the version from Kaggle's Models page instead.
#       Check it out here: 
#       (https://www.kaggle.com/models/google/universal-sentence-encoder)
# -----------------------------------
encoder_link = 'https://kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2'
# encoder_link = 'https://tfhub.dev/google/universal-sentence-encoder/4'

encoder_name = 'universal_sentence_encoder'
encoder_trainable=False # set trainable to False for inference-only 

encoder = get_tfhub_model(encoder_link, encoder_name, model_trainable=encoder_trainable)

<a id='4.2'></a>
### Build Model

In [ ]:
def build_baseline_model(num_classes=5):
    # Define kernel initializer & input layer
    initializer = tf.keras.initializers.HeNormal(seed=CFG.SEED)
    review_input = layers.Input(shape=[], dtype=tf.string, name='review_text_input')
    
    # Generate Embeddings
    review_embedding = encoder(review_input)
    
    # Feed Embeddings to a Bidirectional LSTM
    expand_layer = layers.Lambda(lambda embed: tf.expand_dims(embed, axis=1))(review_embedding)
    bi_lstm = layers.Bidirectional(layers.LSTM(128, kernel_initializer=initializer), 
                                   name='bidirection_lstm')(expand_layer)
    
    # Feed LSTM output to classification head
    dropout_layer = layers.Dropout(0.25)(bi_lstm)
    dense_layer = layers.Dense(64, activation='relu', kernel_initializer=initializer)(dropout_layer)
    output_layer = layers.Dense(num_classes, activation='softmax', 
                                kernel_initializer=initializer, 
                                name='output_layer')(dense_layer)
    
    return tf.keras.Model(inputs=[review_input], 
                          outputs=[output_layer], 
                          name='use_model')

In [ ]:
# Build model
model = build_baseline_model()

# View summary of model
model.summary()

In [ ]:
# Explore model visually
plot_model(
    model, dpi=60,
    show_shapes=True,
    expand_nested=True
)

<center><div style='color:#ffffff;
           display:inline-block;
           padding: 5px 5px 5px 5px;
           border-radius:5px;
           background-color:#03e3fc;
           font-size:100%;'><a href=#toc style='text-decoration: none; color:#03001C;'>⬆️ Back To Top</a></div></center>
           
<a id='5'></a>
# 5 | Train Baseline Model
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>
<br>

To train this model we'll use Categorical Crossentropy as the loss function since this notebook approaches the problem at hand as a classification problem for multiple labels. As for the optimizer, we'll use the Adam optimizer with 0.001 as the (default) learning rate. 

To prevent the occurance of overfitting during training we'll have to make use of TensorFlow's Callback API to implement the EarlyStopping & ReduceLROnPlateau callbacks. The only metrics we'll track during the training of the model will be the loss and accuracy metrics.

**See the following for more information:**
> - **Categorical Crossentropy Loss Function:**
>    - [Understanding Categorical Cross-Entropy Loss, Binary Cross-Entropy Loss, Softmax Loss, Logistic Loss, Focal Loss and all those confusing names](https://gombru.github.io/2018/05/23/cross_entropy_loss/)
>    - [TensorFlow Categorical Crossentropy Loss Implementation](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy)
> - **Adam Optimizer:**
>    - [Academic Paper | Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980)
>     - [TensorFlow Adam Implementation](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)
> - **TensorFlow Callback API:**
>    - [EarlyStopping Implementation](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)
>    - [ReduceLROnPlateau Implementation](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau)
> - **TensorFlow Metrics:**
>    - [TensorFlow Metrics Overview](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [ ]:
def train_model(model, num_epochs, callbacks_list, tf_train_data, 
                tf_valid_data=None, shuffling=False):
    '''
        Trains a TensorFlow model and returns a dict object containing the model metrics history data. 
        
        @params
        - model: (tf.keras.model) -> model to be trained 
        - num_epochs: (int) -> number of epochs to train the model
        - callbacks_list: (list) -> list containing callback fuctions for model
        - tf_train_data: (tf.data.Dataset) -> dataset for model to be train on 
        - tf_valid_data: (tf.data.Dataset) -> dataset for model to be validated on (default=None)
        - shuffling: (bool) -> condition for data shuffling, data is shuffled when True (default=False)
        
        @returns
        - model_history: (dict) -> dictionary containing loss and metrics values tracked during training
    '''
    
    model_history = {}
    
    if tf_valid_data != None:
        model_history = model.fit(tf_train_data,
                                  epochs=num_epochs,
                                  validation_data=tf_valid_data,
                                  validation_steps=int(len(tf_valid_data)),
                                  callbacks=callbacks_list,
                                  shuffle=shuffling)
        
    if tf_valid_data == None:
        model_history = model.fit(tf_train_data,
                                  epochs=num_epochs,
                                  callbacks=callbacks_list,
                                  shuffle=shuffling)
    return model_history

<a id=5.1></a>
### Define Callbacks and Metrics for Model Training

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=4, 
    restore_best_weights=True)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    patience=2,
    factor=0.1,
    verbose=1)

CALLBACKS = [early_stopping_callback, reduce_lr_callback]
METRICS = ['accuracy']

<a id=5.2></a>
### Compile & Train Model

In [ ]:
tf.random.set_seed(CFG.SEED)

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=METRICS
)

print(f'Training {model.name}.')
print(f'Train on {len(train_new_df)} samples, validate on {len(val_df)} samples.')
print('----------------------------------')

model_history = train_model(
    model, CFG.EPOCHS, CALLBACKS, 
    train_ds, val_ds,
    shuffling=False
)

In [ ]:
# Evaluate the model
model_evaluation = model.evaluate(val_ds)

In [ ]:
# Generate model probabilities and associated predictions
train_probabilities = model.predict(train_ds, verbose=1)
train_predictions = tf.argmax(train_probabilities, axis=1)

In [ ]:
# Generate model probabilities and associated predictions
val_probabilities = model.predict(val_ds, verbose=1)
val_predictions = tf.argmax(val_probabilities, axis=1)

<center><div style='color:#ffffff;
           display:inline-block;
           padding: 5px 5px 5px 5px;
           border-radius:5px;
           background-color:#03e3fc;
           font-size:100%;'><a href=#toc style='text-decoration: none; color:#03001C;'>⬆️ Back To Top</a></div></center>
           
<a id='6'></a>
# 6 | Model Performance Evaluation
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>
<br>

Now that the model has trained on the data we need to inspect how well it performs on the validation data. In order to conduct this inspection we need to evaluate the performance of the model on the validation data and record evaluation metrics. Since the approach for this problem is a multi classification problem we'll make use of some well known classification metrics. Hence, we'll make use of the Scikit Learn library to inspect the model. We'll also use the following to inspect the model:

> - [Classification Report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report)
> - [Accuracy Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score)
> - [Precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)
> - [Recall](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score)
> - [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score) 
> - [Matthews Correlation Coefficient](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html)

<a id='6.1'></a>
### Plot Model Training History

In [ ]:
def plot_training_curves(history):
    
    loss = np.array(history.history['loss'])
    val_loss = np.array(history.history['val_loss'])

    accuracy = np.array(history.history['accuracy'])
    val_accuracy = np.array(history.history['val_accuracy'])

    epochs = range(len(history.history['loss']))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    # Plot loss
    ax1.plot(epochs, loss, label='training_loss', marker='o')
    ax1.plot(epochs, val_loss, label='val_loss', marker='o')
    
    ax1.fill_between(epochs, loss, val_loss, where=(loss > val_loss), color='C0', alpha=0.3, interpolate=True)
    ax1.fill_between(epochs, loss, val_loss, where=(loss < val_loss), color='C1', alpha=0.3, interpolate=True)

    ax1.set_title('Loss (Lower Means Better)', fontsize=16)
    ax1.set_xlabel('Epochs', fontsize=14)
    ax1.legend()
    sns.despine();

    # Plot accuracy
    ax2.plot(epochs, accuracy, label='training_accuracy', marker='o')
    ax2.plot(epochs, val_accuracy, label='val_accuracy', marker='o')
    
    ax2.fill_between(epochs, accuracy, val_accuracy, where=(accuracy > val_accuracy), color='C0', alpha=0.3, interpolate=True)
    ax2.fill_between(epochs, accuracy, val_accuracy, where=(accuracy < val_accuracy), color='C1', alpha=0.3, interpolate=True)

    ax2.set_title('Accuracy (Higher Means Better)', fontsize=16)
    ax2.set_xlabel('Epochs', fontsize=14)
    ax2.legend();
    sns.despine();

In [ ]:
# plot model training history 
plot_training_curves(model_history)

<div class="alert alert-block alert-info">
<br>
We observe that overfitting may have occured during the first few expochs. We also observe that the model reached a plateau on the validation loss.
</div>

<a id='6.2'></a>
### Plot Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes='auto', figsize=(10, 10), text_size=12): 
    # Generate confusion matrix 
    cm = confusion_matrix(y_true, y_pred)
    
    # Set plot size
    plt.figure(figsize=figsize)

    # Create confusion matrix heatmap
    disp = sns.heatmap(
        cm, annot=True, cmap='Greens',
        annot_kws={"size": text_size}, fmt='g',
        linewidths=1, linecolor='black', clip_on=False,
        xticklabels=classes, yticklabels=classes)
    
    # Set title and axis labels
    disp.set_title('Confusion Matrix', fontsize=24)
    disp.set_xlabel('Predicted Label', fontsize=20) 
    disp.set_ylabel('True Label', fontsize=20)
    plt.yticks(rotation=0) 

    # Plot confusion matrix
    plt.show()
    
    return

In [ ]:
plot_confusion_matrix(
    val_df.Rating - 1, 
    val_predictions, 
    figsize=(10, 10))

<div class="alert alert-block alert-info">
<br>
The model is able to classify the majority classes. However, the characteristics of a severly imbalanced dataset is present as the model struggles with predicting the minority classes.
</div>

<a id='6.3'></a>
### Generate Classification Report

In [ ]:
print(classification_report(val_df.Rating - 1, val_predictions))

<a id='6.4'></a>
### Record Classification Metrics

In [ ]:
def generate_preformance_scores(y_true, y_pred, y_probabilities):
    
    model_accuracy = accuracy_score(y_true, y_pred)
    top_2_accuracy = top_k_accuracy_score(y_true, y_probabilities, k=2)
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, 
                                                                                 y_pred, 
                                                                                 average="weighted")
    model_matthews_corrcoef = matthews_corrcoef(y_true, y_pred)
    
    print('=============================================')
    print(f'\nPerformance Metrics:\n')
    print('=============================================')
    print(f'accuracy_score:\t\t{model_accuracy:.5f}\n')
    print('_____________________________________________')
    print(f'top_2_accuracy_score:\t{top_2_accuracy:.5f}\n')
    print('_____________________________________________')
    print(f'precision_score:\t{model_precision:.5f}\n')
    print('_____________________________________________')
    print(f'recall_score:\t\t{model_recall:.5f}\n')
    print('_____________________________________________')
    print(f'f1_score:\t\t{model_f1:.5f}\n')
    print('_____________________________________________')
    print(f'matthews_corrcoef:\t{model_matthews_corrcoef:.5f}\n')
    print('=============================================')
    
    preformance_scores = {
        'accuracy_score': model_accuracy,
        'top_2_accuracy_score': top_2_accuracy,
        'precision_score': model_precision,
        'recall_score': model_recall,
        'f1_score': model_f1,
        'matthews_corrcoef': model_matthews_corrcoef
    }
    
    return preformance_scores

In [ ]:
model_performance = generate_preformance_scores(val_df.Rating-1, val_predictions, val_probabilities)

In [ ]:
# Inspect Competition Metric: Mean-Absolute-Error
print('Competition Metric Score')
print('=========================')
print(f'Train MAE:\t{mean_absolute_error(train_new_df.rating_encoded, train_predictions):.5f}')
print(f'Validation MAE:\t{mean_absolute_error(val_df.rating_encoded, val_predictions):.5f}')
print('=========================')

<div class="alert alert-block alert-info">
<br>
The model was able to achive a Matthews Correlation Coefficient of ~0.76 which is decent. A high MCC implies that the model's predictions are statistically of high quality and that the model may generalise to unseen samples. Looking at the Compition Metric, the model seems to score a low validation MAE and we can observe a significant difference between the training and validation MAE scores.
   <br><br>
   <b>Since the model was able to score a high MCC, we should expect a similar LB MAE score as the validation MAE.</b>
</div>

<center><div style='color:#ffffff;
           display:inline-block;
           padding: 5px 5px 5px 5px;
           border-radius:5px;
           background-color:#03e3fc;
           font-size:100%;'><a href=#toc style='text-decoration: none; color:#03001C;'>⬆️ Back To Top</a></div></center>

<a id='7'></a>
# 7 | Generate Submission
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>
<br>

With the model performance evaluation complete, we need to generate the submission file.

<a id='7.1'></a>
### Preprocess Test Reviews

In [ ]:
def predict(model, test_reviews):
    probabilities = model.predict(test_reviews, verbose=1)
    predictions = tf.argmax(probabilities, axis=1)
    return probabilities, predictions

In [ ]:
# Preprocess Test Reviews
test_reviews = test_df['Review'].apply(text_preprocessor)
test_reviews.shape

<a id='7.2'></a>
### Generate Test Predictions

In [ ]:
# Generate Test Predictions
test_probabilities, test_predictions = predict(model, test_reviews)

<a id='7.3'></a>

In [ ]:
# Use the sample_subission dataframe to create the
# submission csv for the test set predictions
submission_df['Rating'] = test_predictions + 1 # Decode labels 

# View first 5 submission samples 
submission_df.head(5)

In [ ]:
# View Test Predictions Ratings Distribution
plt.figure(figsize=(14, 8))
plt.title('Test Prediction Ratings Distribution', fontsize=20)
test_predictions_distribution = submission_df['Rating'].value_counts().sort_values()

sns.barplot(x=test_predictions_distribution.values,
            y=list(test_predictions_distribution.keys()),
            orient="h");
sns.despine();

<div class="alert alert-block alert-info">
<br>
The test predictions resemble the train ratings distribution. However, we observe that the minority classes are overshadowed by the majority classes. This behavior should be addressed in future experiments.
</div>

<a id='7.4'></a>
### Generate Submission.csv

In [ ]:
# Create submission csv
submission_df.to_csv('submission.csv', index=False)

### Get Session Info (Package Requirements)

In [ ]:
! pip -q install session_info

In [ ]:
import session_info
session_info.show(html=False)

<center><div style='color:#ffffff;
           display:inline-block;
           padding: 5px 5px 5px 5px;
           border-radius:5px;
           background-color:#03e3fc;
           font-size:100%;'><a href=#toc style='text-decoration: none; color:#03001C;'>⬆️ Back To Top</a></div></center>

<a id='conclusion'></a>
# <center>Conclusion</center>
<div style="padding: 4px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;"></div>
<br>

In this notebook we built a baseline model to predict the sentiment of company reviews left by customers. We achieved this by using the Universal Sentence Encoder to generate text embeddings which where representative of the review texts, and fed these embeddings to a classification head. We achieved a Matthews Correlation Coefficient of ~0.76, which implies that the model's predictions are statistically of high quality and that the model will generalise to unseen samples.

However, when looking at the model's confusion matrix and classification report we observe the characteristics of a severly imbalanced dataset. The model is unable to correctly predict review ratings which are between 1 and 5. Also, the classification approach might be adding to this bad model behavior as the loss function does not aim to minimize the competition metric (Mean Absolute Error). Therefore, it is recommended that an **ordinal text regression approach** should be followed as this approach will be focuse on minimizing the competition's metric and may achieve better results.

<br>

<div style="color:white;padding: 15px;color:white;margin:10;font-size:120%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://i.postimg.cc/fTDmwnkQ/Miaka.png); background-size: 100% auto;background-position: 0px 0px;">
    <center>
    <h3><span style='color:white'>
        I hope this notebook serves the community well!<br><br>
        Thank you for visiting!</span>🙏</h3>
    </center>
</div>